In [5]:
import numpy as np
import pandas as pd
import os
import cv2
import math
import random
import matplotlib.pyplot as plt
import shutil
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MinMaxScaler
from PIL import Image
import warnings
from pathlib import Path
warnings.filterwarnings("ignore")

In [6]:
def dataframe_from_csv(target):
    return pd.read_csv(target).rename(columns=lambda x: x.strip())

def dataframe_from_csvs(targets):
    return pd.concat([dataframe_from_csv(x) for x in targets])

TEST_DATASET = sorted([x for x in Path("./21/test/").glob("*.csv")])
TRAIN_DATASET = sorted([x for x in Path("./21/train/").glob("*.csv")])
TEST_DF_RAW = dataframe_from_csvs(TEST_DATASET)
TRAIN_DF_RAW = dataframe_from_csvs(TRAIN_DATASET)

# TRAIN_DF_RAW = dataframe_from_csvs(TRAIN_DATASET)
# TRAIN_DF_RAW.reset_index(drop=True,inplace=True)
print(TRAIN_DF_RAW.shape)
TRAIN_DF_RAW.head()

(921603, 84)


,time,P1_B2004,P1_B2016,P1_B3004,P1_B3005,P1_B4002,P1_B4005,P1_B400B,P1_B4022,P1_FCV01D,...,P4_ST_GOV,P4_ST_LD,P4_ST_PO,P4_ST_PS,P4_ST_PT01,P4_ST_TT01,attack,attack_P1,attack_P2,attack_P3
0,2020-07-11 00:00:00,0.10121,1.29784,397.63785,1001.99799,33.6555,100.0,2847.02539,37.14706,100.0,...,16495.0,301.35992,305.03113,0,10052.0,27610.0,0,0,0,0
1,2020-07-11 00:00:01,0.10121,1.29692,397.63785,1001.99799,33.6555,100.0,2839.58520,37.14477,100.0,...,16402.0,297.43567,304.27161,0,10052.0,27610.0,0,0,0,0
2,2020-07-11 00:00:02,0.10121,1.29631,397.63785,1001.99799,33.6555,100.0,2833.26807,37.14325,100.0,...,16379.0,298.66534,303.89179,0,10050.0,27617.0,0,0,0,0
3,2020-07-11 00:00:03,0.10121,1.28685,397.63785,1001.99799,33.6555,100.0,2834.95264,37.11959,100.0,...,16422.0,298.06860,303.67474,0,10052.0,27614.0,0,0,0,0
4,2020-07-11 00:00:04,0.10121,1.28807,397.63785,1001.99799,33.6555,100.0,2832.70654,37.12265,100.0,...,16355.0,296.53137,303.22266,0,10052.0,27621.0,0,0,0,0


In [7]:
TEST_DF_RAW.describe()

,P1_B2004,P1_B2016,P1_B3004,P1_B3005,P1_B4002,P1_B4005,P1_B400B,P1_B4022,P1_FCV01D,P1_FCV01Z,...,P4_ST_GOV,P4_ST_LD,P4_ST_PO,P4_ST_PS,P4_ST_PT01,P4_ST_TT01,attack,attack_P1,attack_P2,attack_P3
count,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000,...,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000
mean,0.091431,1.389921,403.798729,1066.074738,32.285249,58.924257,1741.935584,36.031480,53.314444,53.103314,...,17923.855068,332.192397,332.465165,8.918421,10050.187750,27608.471828,0.022256,0.017505,0.004637,0.002383
std,0.018979,0.134062,11.981150,51.409852,0.571706,42.214697,1200.694504,0.670232,43.104898,43.098623,...,1828.472994,37.394611,33.982635,19.141153,24.580383,24.517653,0.147515,0.131143,0.067936,0.048758
min,0.030110,0.959140,374.638610,902.981140,31.379050,0.000000,25.025980,34.225440,0.000000,0.260930,...,12660.000000,231.119810,235.622800,0.000000,9695.000000,27519.000000,0.000000,0.000000,0.000000,0.000000
25%,0.098290,1.312640,397.637850,1018.359010,32.000000,10.804490,305.078120,35.577770,13.731810,13.276670,...,16879.000000,307.526790,313.114870,0.000000,10051.000000,27592.000000,0.000000,0.000000,0.000000,0.000000
50%,0.099510,1.386060,400.835820,1085.589360,32.000000,74.562280,2477.326420,35.887980,38.567150,38.652040,...,17859.000000,331.778080,331.217470,0.000000,10053.000000,27623.000000,0.000000,0.000000,0.000000,0.000000
75%,0.101090,1.464340,407.549830,1104.476070,32.615000,100.000000,2831.723880,36.399530,100.000000,99.708420,...,18919.000000,355.052860,350.929570,0.000000,10053.000000,27627.000000,0.000000,0.000000,0.000000,0.000000
max,0.101930,2.009630,448.763430,1121.941160,33.655500,100.000000,2860.977300,38.455970,100.000000,100.000000,...,26450.000000,496.455440,491.500310,50.000000,10330.000000,27627.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
TRAIN_DF_RAW.describe()

,P1_B2004,P1_B2016,P1_B3004,P1_B3005,P1_B4002,P1_B4005,P1_B400B,P1_B4022,P1_FCV01D,P1_FCV01Z,...,P4_ST_GOV,P4_ST_LD,P4_ST_PO,P4_ST_PS,P4_ST_PT01,P4_ST_TT01,attack,attack_P1,attack_P2,attack_P3
count,921603.000000,921603.000000,921603.000000,921603.000000,921603.000000,921603.000000,921603.000000,921603.000000,921603.000000,921603.000000,...,921603.000000,921603.000000,921603.000000,921603.000000,921603.000000,921603.000000,921603.0,921603.0,921603.0,921603.0
mean,0.093568,1.391266,403.386064,1061.048787,32.316926,60.525107,1778.475870,36.061175,55.000463,54.823837,...,17912.961261,332.002897,332.268971,7.810684,10050.575157,27609.032692,0.0,0.0,0.0,0.0
std,0.017998,0.135774,15.418150,61.183317,0.578673,42.181297,1193.352273,0.671451,43.272894,43.299978,...,1852.143216,37.853169,34.450944,18.152900,17.784770,23.917322,0.0,0.0,0.0,0.0
min,0.029780,0.947290,369.756010,890.078430,31.413430,0.000000,25.025980,34.202320,0.000000,0.260930,...,12625.000000,229.872010,233.561220,0.000000,9914.000000,27530.000000,0.0,0.0,0.0,0.0
25%,0.098370,1.313060,396.570710,1005.136960,32.000000,12.154330,361.684420,35.606770,13.990915,13.589480,...,16851.000000,307.110840,312.554260,0.000000,10051.000000,27593.000000,0.0,0.0,0.0,0.0
50%,0.099770,1.386460,404.364040,1091.510010,32.000000,80.049410,2660.262450,35.900580,46.639160,46.731560,...,17840.000000,331.199340,330.873840,0.000000,10053.000000,27625.000000,0.0,0.0,0.0,0.0
75%,0.100870,1.464970,406.226260,1106.839110,32.543950,100.000000,2829.898930,36.392590,100.000000,99.916080,...,18913.000000,355.052860,350.821050,0.000000,10053.000000,27627.000000,0.0,0.0,0.0,0.0
max,0.101960,2.052300,447.834380,1121.941160,33.655500,100.000000,2857.545650,38.650930,100.000000,100.000000,...,26927.000000,499.620180,498.860660,50.000000,10330.000000,27629.000000,0.0,0.0,0.0,0.0


In [9]:
TIMESTAMP_FIELD = "time"
ATTACK_FIELD = "attack"
NOT_VALID_FIELD = ["time", "attack", "attack_P1", "attack_P2", "attack_P3"]
ATTACK_DF = TEST_DF_RAW["attack"]
VALID_COLUMNS_IN_TRAIN_DATASET = TRAIN_DF_RAW.columns.drop(NOT_VALID_FIELD)
TAG_MIN = TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET].min()
TAG_MAX = TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET].max()
TAG_MIN_TEST = TEST_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET].min()
TAG_MAX_TEST = TEST_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET].max()

def normalize(df, TAG_MIN, TAG_MAX):
    ndf = df.copy()
    for c in df.columns:
        if TAG_MIN[c] == TAG_MAX[c]:
            ndf[c] = df[c] - TAG_MIN[c]
        else:
            ndf[c] = (df[c] - TAG_MIN[c]) / (TAG_MAX[c] - TAG_MIN[c])
    return ndf

# Min-Max Normalize
TRAIN_DF = normalize(TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET], TAG_MIN, TAG_MAX).ewm(alpha=0.9).mean()
TRAIN_DF

TEST_DF = normalize(TEST_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET], TAG_MIN_TEST, TAG_MAX_TEST).ewm(alpha=0.9).mean()
TEST_DF

,P1_B2004,P1_B2016,P1_B3004,P1_B3005,P1_B4002,P1_B4005,P1_B400B,P1_B4022,P1_FCV01D,P1_FCV01Z,...,P4_HT_PO,P4_HT_PS,P4_LD,P4_ST_FD,P4_ST_GOV,P4_ST_LD,P4_ST_PO,P4_ST_PS,P4_ST_PT01,P4_ST_TT01
0,0.997911,0.598359,0.393255,0.376290,0.534262,1.0,0.992457,0.610954,1.000000,0.999159,...,0.821653,0.0,0.867066,0.534560,0.566280,0.584719,0.565482,0.0,0.549606,0.444444
1,0.997911,0.597961,0.393255,0.376290,0.534262,1.0,0.993672,0.610707,1.000000,0.999159,...,0.824774,0.0,0.867618,0.510534,0.567598,0.584843,0.564454,0.0,0.543880,0.419192
2,0.997911,0.604734,0.393255,0.376290,0.534262,1.0,0.994316,0.614910,1.000000,0.999159,...,0.828149,0.0,0.876651,0.501044,0.575230,0.596521,0.569646,0.0,0.539108,0.425259
3,0.997911,0.607358,0.393255,0.376290,0.534262,1.0,0.991930,0.616543,1.000000,0.999159,...,0.830846,0.0,0.877181,0.488210,0.577879,0.594978,0.572132,0.0,0.542888,0.384188
4,0.997911,0.608666,0.393255,0.376290,0.534262,1.0,0.993607,0.617355,1.000000,0.999090,...,0.834206,0.0,0.880873,0.482995,0.584996,0.597461,0.577151,0.0,0.546100,0.413419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92396,0.431774,0.469980,0.579329,0.395145,0.272771,0.0,0.002518,0.414580,0.036732,0.031439,...,0.794300,0.0,0.745532,0.464867,0.478317,0.489185,0.475260,0.0,0.562005,1.000000
92397,0.431774,0.470812,0.579329,0.395145,0.272771,0.0,0.002773,0.415094,0.036894,0.031439,...,0.794922,0.0,0.747946,0.466115,0.478383,0.485546,0.475486,0.0,0.562185,1.000000
92398,0.431774,0.471289,0.579329,0.395145,0.272771,0.0,0.003114,0.415388,0.036678,0.031439,...,0.795712,0.0,0.748309,0.464867,0.477802,0.487758,0.475445,0.0,0.559368,1.000000
92399,0.431774,0.471594,0.579329,0.395145,0.272771,0.0,0.002518,0.415581,0.036422,0.031439,...,0.796517,0.0,0.747132,0.494566,0.479506,0.477184,0.477285,0.0,0.563338,1.000000


In [10]:
TEST_DF = TEST_DF * 255
TRAIN_DF = TRAIN_DF * 255

In [11]:
TRAIN_DF.describe()

,P1_B2004,P1_B2016,P1_B3004,P1_B3005,P1_B4002,P1_B4005,P1_B400B,P1_B4022,P1_FCV01D,P1_FCV01Z,...,P4_HT_PO,P4_HT_PS,P4_LD,P4_ST_FD,P4_ST_GOV,P4_ST_LD,P4_ST_PO,P4_ST_PS,P4_ST_PT01,P4_ST_TT01
count,921603.000000,921603.000000,921603.000000,921603.000000,921603.000000,921603.000000,921603.000000,921603.000000,921603.000000,921603.000000,...,9.216030e+05,921603.000000,921603.000000,921603.000000,921603.000000,921603.000000,921603.000000,921603.000000,9.216030e+05,921603.000000
mean,225.351223,102.455147,109.834056,188.031244,102.758439,154.339023,157.855823,106.551930,140.251180,139.499408,...,9.983786e+01,39.834489,128.143001,130.020198,94.282625,96.546998,94.875723,39.834489,8.371795e+01,203.569062
std,63.585395,31.328553,50.354828,67.288622,65.814936,107.562246,107.429725,38.487294,110.345309,110.703110,...,9.250499e+01,92.579174,57.534365,3.804619,33.019074,35.773726,33.110346,92.579174,1.088087e+01,61.552744
min,0.000000,0.050784,0.000000,0.000000,0.000000,0.000000,0.038529,0.107604,0.000000,0.003901,...,4.854581e-163,0.000000,0.065810,11.754474,0.121348,0.340854,0.060063,0.000000,2.522439e-21,5.633240
25%,242.317124,84.412747,87.575451,126.540066,66.713060,30.994489,30.359073,80.493857,35.731882,34.101651,...,5.856847e-01,0.000000,82.514497,127.609157,75.361621,73.018428,75.946845,0.000000,8.354928e+01,161.981920
50%,247.263092,101.344827,113.028072,221.532166,66.713060,204.126723,237.240466,97.344476,118.940825,118.810118,...,8.554078e+01,0.000000,127.030553,129.992975,92.974041,95.794782,93.533334,0.000000,8.519881e+01,243.708791
75%,251.149210,119.450233,119.109988,238.390937,128.578769,255.000000,252.518257,125.544577,255.000000,254.785444,...,1.962308e+02,0.000000,173.735930,132.449187,112.109947,118.332987,112.700281,0.000000,8.520433e+01,249.848485
max,255.000000,254.830744,255.000000,255.000000,255.000000,255.000000,254.867313,254.966777,255.000000,255.000000,...,2.524774e+02,255.000000,254.859390,242.730117,254.900040,254.893862,254.921267,255.000000,2.550000e+02,255.000000


In [12]:
TEST_DF.describe()

,P1_B2004,P1_B2016,P1_B3004,P1_B3005,P1_B4002,P1_B4005,P1_B400B,P1_B4022,P1_FCV01D,P1_FCV01Z,...,P4_HT_PO,P4_HT_PS,P4_LD,P4_ST_FD,P4_ST_GOV,P4_ST_LD,P4_ST_PO,P4_ST_PS,P4_ST_PT01,P4_ST_TT01
count,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000,...,4.020050e+05,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000,402005.000000
mean,217.722481,104.569366,100.314975,189.938178,101.509283,150.256927,154.379288,108.861145,135.951901,135.100664,...,9.700182e+01,45.483949,128.324583,124.064068,97.337428,97.135509,96.510258,45.483949,142.634450,211.252888
std,67.386272,32.538815,41.216773,59.871644,64.040423,107.647386,107.959913,40.397778,109.916888,110.188274,...,8.942812e+01,97.618877,57.524974,3.997883,33.807148,35.927637,33.862703,97.618877,9.854367,57.841029
min,0.000000,0.103804,0.901222,0.000000,0.000000,0.000000,0.053586,0.007499,0.000000,0.003901,...,1.029839e-52,0.000000,0.226876,12.242662,0.136279,0.316706,0.075182,0.000000,0.401575,0.801778
25%,242.076023,85.808995,79.120680,134.368625,69.556656,27.552267,25.164770,81.519857,35.063156,33.396281,...,8.776391e-01,0.000000,83.480649,121.530004,78.023318,73.427093,77.222659,0.000000,142.934286,172.383741
50%,246.407686,103.640754,90.122155,212.664833,69.556656,190.137634,220.491938,100.206915,98.324355,98.098804,...,8.358973e+01,0.000000,126.571422,124.042550,96.138984,96.779837,95.276495,0.000000,143.759767,246.197799
75%,252.017544,122.623841,113.219312,234.660223,138.446814,255.000000,252.377326,131.041133,255.000000,254.254420,...,1.902935e+02,0.000000,174.004292,126.608045,115.746373,119.102025,114.901239,0.000000,143.763780,255.000000
max,255.000000,254.868243,255.000000,255.000000,255.000000,255.000000,254.773522,254.924248,255.000000,255.000000,...,2.507865e+02,255.000000,254.768618,242.271044,254.806400,254.924276,254.974589,255.000000,255.000000,255.000000


In [13]:
import torch

# 이미지 사이즈 18(특성) * 18 * 3(RGB)
# 윈도우 사이즈 54

window_size = 54
label_size = 20000

def sliding_window_unsupervised(df, window_size, feature_columns):
    # 주어진 특징 열만을 사용하여 슬라이딩 윈도우를 생성
    data = df[feature_columns].values
    num_samples = len(df) - window_size + 1
    features = np.empty((num_samples, window_size * data.shape[1]), dtype=np.float32)

    for i in range(num_samples):
        features[i] = data[i:i + window_size].flatten()

    return features


In [18]:
print(TEST_DF.columns)


Index(['P1_B2004', 'P1_B2016', 'P1_B3004', 'P1_B3005', 'P1_B4002', 'P1_B4005',
       'P1_B400B', 'P1_B4022', 'P1_FCV01D', 'P1_FCV01Z', 'P1_FCV02D',
       'P1_FCV02Z', 'P1_FCV03D', 'P1_FCV03Z', 'P1_FT01', 'P1_FT01Z', 'P1_FT02',
       'P1_FT02Z', 'P1_FT03', 'P1_FT03Z', 'P1_LCV01D', 'P1_LCV01Z', 'P1_LIT01',
       'P1_PCV01D', 'P1_PCV01Z', 'P1_PCV02D', 'P1_PCV02Z', 'P1_PIT01',
       'P1_PIT02', 'P1_PP01AD', 'P1_PP01AR', 'P1_PP01BD', 'P1_PP01BR',
       'P1_PP02D', 'P1_PP02R', 'P1_STSP', 'P1_TIT01', 'P1_TIT02', 'P2_24Vdc',
       'P2_ASD', 'P2_AutoGO', 'P2_CO_rpm', 'P2_Emerg', 'P2_HILout', 'P2_MSD',
       'P2_ManualGO', 'P2_OnOff', 'P2_RTR', 'P2_SIT01', 'P2_SIT02',
       'P2_TripEx', 'P2_VT01', 'P2_VTR01', 'P2_VTR02', 'P2_VTR03', 'P2_VTR04',
       'P2_VXT02', 'P2_VXT03', 'P2_VYT02', 'P2_VYT03', 'P3_FIT01', 'P3_LCP01D',
       'P3_LCV01D', 'P3_LH', 'P3_LIT01', 'P3_LL', 'P3_PIT01', 'P4_HT_FD',
       'P4_HT_LD', 'P4_HT_PO', 'P4_HT_PS', 'P4_LD', 'P4_ST_FD', 'P4_ST_GOV',
       'P4_ST_L

In [19]:
window_size = 54
image_size = (18, 18, 3)
image_path = "train/0/"
csv_path = "train/0/labels.csv"
feature_columns = ['P1_B2016', 'P4_ST_PO', 'P4_LD', 'P1_PIT01', 'P4_ST_PT01', 'P1_FT03', 'P1_FT03Z',
                   'P1_PCV01D', 'P1_PCV01Z', 'P3_LIT01', 'P1_LCV01D', 'P1_LCV01Z', 'P1_FT01',
                   'P1_FT01Z', 'P3_LCV01D', 'P4_HT_PO', 'P4_HT_LD', 'P4_LD']

attack_colum = ['attack']

# 슬라이딩 윈도우 적용
TrainD = sliding_window_unsupervised(TRAIN_DF, window_size, feature_columns)

TestD = sliding_window_unsupervised(TEST_DF, window_size, feature_columns)

AttackD = sliding_window_unsupervised(TEST_DF_RAW, window_size, attack_colum)

In [20]:
TrainD = np.array(TrainD)
print("Windows shape:", TrainD.shape)

TestD = np.array(TestD)
print("Windows shape:", TestD.shape)

AttackD = np.array(AttackD)
print("Windows shape:", AttackD.shape)

Windows shape: (921550, 972)
Windows shape: (401952, 972)
Windows shape: (401952, 54)


In [21]:
import numpy as np
import pandas as pd

# AttackD는 슬라이딩 윈도우를 적용한 결과로 윈도우별 attack 값을 가지고 있다고 가정합니다.

# 각 윈도우에서 하나라도 attack 값이 1인 경우 해당 윈도우의 라벨을 1로 설정, 그렇지 않으면 0으로 설정
labels = np.where(np.any(AttackD == 1, axis=1), 1, 0)

# 라벨을 DataFrame으로 변환
labels_df = pd.DataFrame(labels, columns=['Label'])

# CSV 파일로 저장
labels_df.to_csv('attack_labels.csv', index=False)

print("CSV 파일로 라벨이 저장되었습니다.")


CSV 파일로 라벨이 저장되었습니다.


In [35]:
# 설정값
window_size = 54
image_size = (18, 18, 3)
image_path = "train/"

# 이미지 저장 폴더 생성
os.makedirs(image_path, exist_ok=True)

count = 0
labels = []

for i in range(len(TrainD)):
    window = TrainD[i]
    
    # 윈도우 데이터를 18x18x3 이미지로 변환
    image_data = np.zeros(image_size, dtype=np.uint8)
    flattened_window = window[:972]  # 필요한 데이터 길이는 972 (18*18*3)
    
    # R 채널에 값 할당
    for j in range(18):
        for k in range(18):
            image_data[j, k, 0] = flattened_window[j * 18 + k]
    
    # G 채널에 값 할당
    for j in range(18):
        for k in range(18):
            image_data[j, k, 1] = flattened_window[324 + j * 18 + k]
    
    # B 채널에 값 할당
    for j in range(18):
        for k in range(18):
            image_data[j, k, 2] = flattened_window[648 + j * 18 + k]
    
    # 이미지 저장
    new_image = Image.fromarray(image_data)
    image_file_name = f"image_{count}.png"
    new_image.save(os.path.join(image_path, image_file_name))
    
    count += 1

In [23]:
# 설정값
window_size = 54
image_size = (18, 18, 3)
image_path = "test/"

# 이미지 저장 폴더 생성
os.makedirs(image_path, exist_ok=True)

count = 0
labels = []

for i in range(len(TestD)):
    window = TestD[i]
    
    # 윈도우 데이터를 18x18x3 이미지로 변환
    image_data = np.zeros(image_size, dtype=np.uint8)
    flattened_window = window[:972]  # 필요한 데이터 길이는 972 (18*18*3)
    
    # R 채널에 값 할당
    for j in range(18):
        for k in range(18):
            image_data[j, k, 0] = flattened_window[j * 18 + k]
    
    # G 채널에 값 할당
    for j in range(18):
        for k in range(18):
            image_data[j, k, 1] = flattened_window[324 + j * 18 + k]
    
    # B 채널에 값 할당
    for j in range(18):
        for k in range(18):
            image_data[j, k, 2] = flattened_window[648 + j * 18 + k]
    
    # 이미지 저장
    new_image = Image.fromarray(image_data)
    image_file_name = f"image_{count}.png"
    new_image.save(os.path.join(image_path, image_file_name))
    
    count += 1

In [33]:
import os
from PIL import Image

def get_224(folder, dstdir):
    imgfilepaths = []
    for root, dirs, imgs in os.walk(folder):
        for thisimg in imgs:
            thisimg_path = os.path.join(root, thisimg)
            imgfilepaths.append(thisimg_path)
    
    for thisimg_path in imgfilepaths:
        # Extract the relative path from the source folder and use it in the destination folder
        relative_path = os.path.relpath(thisimg_path, folder)
        new_file_path = os.path.join(dstdir, relative_path)
        
        # Create the directory if it doesn't exist
        dir_name = os.path.dirname(new_file_path)
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
        
        # Open and resize the image
        img = Image.open(thisimg_path)
        img = img.resize((224, 224), Image.LANCZOS)
        
        # Save the resized image to the new directory
        img.save(new_file_path)
    
    print(f'Finished resizing images from {folder} and saved to {dstdir}')

# Example usage
# get_224('path_to_your_18x18_images', 'path_to_save_224x224_images')


In [36]:
DATA_DIR2_224='./train_224/'
get_224(folder='./train/',dstdir=DATA_DIR2_224)

Finished resizing images from ./train/ and saved to ./train_224/


In [37]:
DATA_DIR2_224='./test_224/'
get_224(folder='./test/',dstdir=DATA_DIR2_224)

Finished resizing images from ./test/ and saved to ./test_224/


In [2]:
#early stopping을 위해 val_dataset 생성 (train의 20%)
import os
import random
import shutil

# 원본 이미지들이 있는 폴더 경로와 validation 이미지들을 이동시킬 폴더 경로를 설정합니다.
train_dir = 'train_224'
val_dir = 'val_224'

# validation 폴더가 없으면 생성합니다.
if not os.path.exists(val_dir):
    os.makedirs(val_dir)

# train_224 폴더 안에 있는 모든 파일 목록을 가져옵니다.
all_images = os.listdir(train_dir)

# 전체 이미지 중 20%를 무작위로 선택합니다.
val_images = random.sample(all_images, int(len(all_images) * 0.2))

# 선택된 이미지들을 train_224에서 val_224로 이동합니다.
for image in val_images:
    src_path = os.path.join(train_dir, image)
    dst_path = os.path.join(val_dir, image)
    shutil.move(src_path, dst_path)

print(f"{len(val_images)} images have been moved to {val_dir}.")

36862 images have been moved to val_224.
